# Classification de mots pour des annonces immobilières

### Le but de ce projet est de récuperer les informations importantes d'une annonce immobilière grâce à un classifieur de mots pour des annonces immobilières.

Les informations à récupérer sont : 
+ M2: surface en m2
+ N_PIECES : nombre de pièces
+ N_CHAMBRES: nombre de chambres
+ VILLE:
+ QUARTIER: nom du quartier
+ ADRESSE: nom du la rue (avec le numero si indiqué)
+ TRANSPORTS_PROXIMITE: transports à proximité
+ ANNEE_CONSTRUCTION: annee de construction de l'immeuble
+ CODE_POSTAL: code postal (92130)
+ LOYER_CC: montant du loyer charges comprises
+ LOYER_HC: montant du loyer hors charges
+ CHARGES_LOCATAIRE_MOIS: montant des charges mensuelles
+ DEPOT_GARANTIE: montant du depot de garantie
+ N_ETAGE:numero etage
+ AVEC_ASCENSEUR:
+ DATE_DISPO:
+ TYPE_CHAUFFAGE: individuel /collectif
+ TYPE_LOCATION: meublé ou non meublé
+ PARKING :
+ EXTERIEUR : présence d'un jardin/balcon/terrasse
+ COPROPRIETE :
+ HONORAIRE : montant des honoraires de l'agence
+ STOCKAGE : présence d'une cave/box ou autre élément de stockage

Pour cela nous allons utiliser le NER tagging et des techniques features-based.

# Sommaire

* I/ Importation de la base de données
* II/ Création des features
* III/ Classification avec le modele CRF 
    + A/ Creation base de donnée pour CRF
    + B/ Apprentissage et Validation du modele CRF
        + 1/ Apprentissage et Validation simple du modele CRF
        + 2/ Apprentissage après optimisation des hyperparametres et k-cross Validation sur le train et validation du modele CRF sur le test
* IV/ Interpretation avec le modele CRF

# Importation des librairies

In [1]:
import json
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
import unicodedata
from difflib import SequenceMatcher
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter
import eli5
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sujiv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.


# I/ Importation de la base de données

### Fonctions

In [2]:
def from_ad_to_dataframe(line, nb_line):
    """Fonction qui à partir d'une ligne d'un fichier Json de créer une dataframe à 
    trois colonnes. La première colonne correspond au numéro de l'annonce, la seconde 
    contient les mots de l'annonce et la troisième les positions du mot dans l'annonce.""" 
    Vect_word=(word_tokenize(eval(line.strip().replace('\xa0',' '))["text"])) # Tokenisation
    nb_sent_list=list(map(int, nb_line*np.ones(len(Vect_word)))) # Numéro annonce
    # Position
    offset = 0                                                                  
    list_pos=list()
    for token in Vect_word:
        offset = eval(line.strip().replace('\xa0',' '))["text"].find(token, offset)
        list_pos.append([offset, offset+len(token)])
        offset += len(token)
    # Creation de la dataframe
    data={'Ad#':nb_sent_list,'Words':Vect_word,'Pos':list_pos}
    df=pd.DataFrame(data)
    return df

def clean_text(text):
    """Fonction qui permet de corriger les annotations qui surlignent un espace blanc au début 
    ou à la fin de l'annotations sous Doccano"""
    if text[0]==' ':      
        if text[-1]==' ':
            return 3
        return 1
    elif text[-1]==' ':
        return 2
    else :
        return 0
    
def from_line_to_list_label(line):
    """Fonction qui permet de sortir les informations des labels (text, label et positions)
    à partir d'une ligne du fichier json"""
    list_word_label=list()
    for i in range(len(eval(line)["labels"])):
        start=eval(line)["labels"][i][0]  # position de depart
        end=eval(line)["labels"][i][1]    # position d'arrivee
        label=eval(line)["labels"][i][2]  # label
        # Distinction des cas au fonction de la fonction clean_text
        # on supprime les \xa0 de nos annonces 
        if (clean_text(eval(line.strip().replace('\xa0',' '))["text"][start:end])==0):
            list_word_label.append([eval(line.strip().replace('\xa0',' '))["text"][start:end],label,start,end])
        elif (clean_text(eval(line.strip().replace('\xa0',' '))["text"][start:end])==1):
            list_word_label.append([eval(line.strip().replace('\xa0',' '))["text"][(start+1):end],label,start+1,end])
        elif (clean_text(eval(line.strip().replace('\xa0',' '))["text"][start:end])==2):
            list_word_label.append([eval(line.strip().replace('\xa0',' '))["text"][start:(end-1)],label,start,end-1])
        else:
            list_word_label.append([eval(line.strip().replace('\xa0',' '))["text"][(start+1):(end-1)],label,start+1,end-1])
    return list_word_label

def column_tag(vect_word,list_word_pos_label):
    """Creation de la colonne contenant les labels pour chaque mot d'une annonce avec la convention Inside–outside–beginning tagging"""
    list_tag=["O"]*len(vect_word["Pos"])
    for i in range(len(vect_word["Pos"])):
        for elmt in list_word_pos_label:
            if vect_word["Pos"][i][0]==elmt[2] and vect_word["Pos"][i][1]<=elmt[3]:
                list_tag[i]="B-"+elmt[1]
            elif vect_word["Pos"][i][0]>elmt[2] and vect_word["Pos"][i][1]<=elmt[3]:
                list_tag[i]="I-"+elmt[1]
    return list_tag

### Importation et creation de la base de données à partir des fichiers Json1 de Doccano

In [3]:
cnt = 1 # Numéro annonce
for i in range(1,6): # 5 fichier Json
    with open('data/doccano/bdd'+str(i)+'.json1', encoding="utf-8") as fp: #ouverture fichier
        line = fp.readline() # lecture de la ligne
        # Modification de la dataframe
        if i==1: # Creation initiale de la dataframe 
            df=from_ad_to_dataframe(line.replace('null','"null"'),cnt)
            list_word_pos_label=from_line_to_list_label(line.replace('null','"null"'))
            list_tag=column_tag(df,list_word_pos_label)
            df["Tag"]=list_tag
        else :
            df_ad=from_ad_to_dataframe(line.replace('null','"null"'),cnt)
            list_word_pos_label=from_line_to_list_label(line.replace('null','"null"'))
            list_tag=column_tag(df_ad,list_word_pos_label)
            df_ad["Tag"]=list_tag
            df=df.append(df_ad, ignore_index = True)
        while line: # pour toutes les lignes
            if cnt!=1:
                df_ad=from_ad_to_dataframe(line.replace('null','"null"'),cnt)
                list_word_pos_label=from_line_to_list_label(line.replace('null','"null"'))
                list_tag=column_tag(df_ad,list_word_pos_label)
                df_ad["Tag"]=list_tag
                df=df.append(df_ad, ignore_index = True)
            line = fp.readline()
            cnt += 1

In [4]:
df.loc[df['Ad#']==482,:].head() # On affiche une partie de la dataframe pour l'annonce 482

,Ad#,Words,Pos,Tag
41057,482,24,"[0, 2]",B-ADRESSE
41058,482,rue,"[3, 6]",I-ADRESSE
41059,482,du,"[7, 9]",I-ADRESSE
41060,482,Capitaine,"[10, 19]",I-ADRESSE
41061,482,Ferber-,"[20, 27]",O


A ce niveau la dataframe contient donc une colonne qui indique le numéro de l'annonce, une colonne avec les mots, une colonne pour les postions du mots dans l'annonce et le label/tag avec les convientions IOB

# II/ Creation des features

Nous allons créer les features suivantes : 
+ le mot contient 4 caractères ou moins 
+ le mot est un nombre
+ le mot commence par une lettre majuscule 
+ le mot est en majuscule
+ le mot contient un symbole 
+ le mot contient des chiffres et des lettres 
+ le mot contient un mot clé (nous avons une liste de mots clés que nous avons choisi judicieusement)
+ le mot precedent et reconnaitre si c'est un mot clé
+ le mot precedent le mot precedent et reconnaitre si c'est un mot clé
+ le mot suivant et reconnaitre si c'est un mot clé
+ le mot suivant le mot suivant et reconnaitre si c'est un mot clé

### Fonctions

In [5]:
# Feature mot court
def is_small_word(Vect_word):
    list_small=list()
    for word in Vect_word:      
        if(len(word)<=4):
            list_small.append(1)
        else:
            list_small.append(0)
    return list_small

# Feature le mot est un nombre (decimal ou non)
def is_number(Vect_word):
    list_number=list()
    for word in Vect_word:      
        word=word.replace(",","").replace(".","") # on peut ecrire un nombre decimal avec un point ou une virgule
        try:
            float(word)
            list_number.append(1)
        except ValueError:
            list_number.append(0)
    return list_number

# Feature premiere lettre en majuscule
def is_first_letter_upper(Vect_word):
    list_upper=list()
    for word in Vect_word:
        list_upper.append(int(word[0].isupper()))
    return list_upper

# Feature mot en majuscule
def is_all_upper(Vect_word):
    list_upper=list()
    for word in Vect_word:
        list_upper.append(int(word.isupper()))
    return list_upper

# Feature symbole dans le mot
def symbole_in_word(Vect_word):
    list_symbole=list()
    for word in Vect_word:
        list_symbole.append( int( not( word.isalpha() or word.isnumeric() ) ) ) # pas un chiffre, pas une lettre donc un symbole
    return list_symbole

#Feature nombre et lettre dans le mot
def is_number_and_letter(Vect_word):
    list_number_and_letter = list()
    for word in Vect_word:
        numeric = 0
        alpha = 0
        for c in word:
            if c.isnumeric():
                numeric=1
            if c.isalpha():
                alpha=1
        list_number_and_letter.append(alpha*numeric)
    return(list_number_and_letter)

#Feature mot clé

def strip_accents(text):
    """Fonction qui retire les accents du texte"""
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return text

def is_key_word(Vect_word):
    keywords = ["chambres","pieces","m2","m","loyer","cc","hc","rue","avenue","quartier","euro","eur","etage","€","individuel","collectif",
                "meuble","jardin","balcon","terasse","stationnement","parking","cave","box","immediatement","suite"]
    list_keyword = list()
    for word in Vect_word:
        test = 0
        #remove all accents
        s = strip_accents(word)
        #put it in lower case
        s = s.lower()
        #use sequencematcher
        for key in keywords:
            if(SequenceMatcher(None,s,key).ratio() > 0.7): # 0.7 est le threshold
                test = 1
                break
        list_keyword.append(test)
    return(list_keyword)

On complete la dataframe

In [6]:
%%time

df["is_small_word"]=is_small_word(df['Words'])
df["is_number"]=is_number(df['Words'])
df["is_first_letter_upper"]=is_first_letter_upper(df['Words'])
df["is_all_upper"]=is_all_upper(df['Words'])
df["symbole_in_word"]=symbole_in_word(df['Words'])
df["is_number_and_letter"]=is_number_and_letter(df['Words'])
df["is_key_word"]=is_key_word(df['Words'])

Wall time: 1min 2s


In [7]:
df.loc[df['Ad#']==482,:].head() # On affiche une partie de la dataframe pour l'annonce 482

,Ad#,Words,Pos,Tag,is_small_word,is_number,is_first_letter_upper,is_all_upper,symbole_in_word,is_number_and_letter,is_key_word
41057,482,24,"[0, 2]",B-ADRESSE,1,1,0,0,0,0,0
41058,482,rue,"[3, 6]",I-ADRESSE,1,0,0,0,0,0,1
41059,482,du,"[7, 9]",I-ADRESSE,1,0,0,0,0,0,0
41060,482,Capitaine,"[10, 19]",I-ADRESSE,0,0,1,0,0,0,0
41061,482,Ferber-,"[20, 27]",O,0,0,1,0,1,0,0


In [8]:
# Features mots precedents, mots suivants, et reconnaitre si ce sont des mots clés
Number_ad=df['Ad#'].iloc[-1]

list_prev_word=list()
list_2prev_word=list()
list_next_word=list()
list_2next_word=list()

list_prev_key=list()
list_2prev_key=list()
list_next_key=list()
list_2next_key=list()

for i in range(Number_ad):
    Vect_ad_word=df.loc[df['Ad#']==i+1,:]["Words"]
    Vect_ad_key=df.loc[df['Ad#']==i+1,:]["is_key_word"]
    for j in range(len(Vect_ad_word)):
        # Pour chaque annonce, on fait attention aux deux premiers et deux derniers mots  
        if j==0:
            list_prev_word.append("__Start1__")
            list_2prev_word.append("__Start2__")
            list_next_word.append(Vect_ad_word.iloc[j+1])
            list_2next_word.append(Vect_ad_word.iloc[j+2])
            
            list_prev_key.append(0)
            list_2prev_key.append(0)
            list_next_key.append(Vect_ad_key.iloc[j+1])
            list_2next_key.append(Vect_ad_key.iloc[j+2])
            
        elif j==1:
            list_prev_word.append(Vect_ad_word.iloc[j-1])
            list_2prev_word.append("__Start1__")
            list_next_word.append(Vect_ad_word.iloc[j+1])
            list_2next_word.append(Vect_ad_word.iloc[j+2])
            
            list_prev_key.append(Vect_ad_key.iloc[j-1])
            list_2prev_key.append(0)
            list_next_key.append(Vect_ad_key.iloc[j+1])
            list_2next_key.append(Vect_ad_key.iloc[j+2])
        
        elif j==len(Vect_ad_word)-2:
            list_prev_word.append(Vect_ad_word.iloc[j-1])
            list_2prev_word.append(Vect_ad_word.iloc[j-2])
            list_next_word.append(Vect_ad_word.iloc[j+1])
            list_2next_word.append("__End1__")
            
            list_prev_key.append(Vect_ad_key.iloc[j-1])
            list_2prev_key.append(Vect_ad_key.iloc[j-2])
            list_next_key.append(Vect_ad_key.iloc[j+1])
            list_2next_key.append(0)
        
        elif j==len(Vect_ad_word)-1:
            list_prev_word.append(Vect_ad_word.iloc[j-1])
            list_2prev_word.append(Vect_ad_word.iloc[j-2])
            list_next_word.append("__End1__")
            list_2next_word.append("__End2__")
            
            list_prev_key.append(Vect_ad_key.iloc[j-1])
            list_2prev_key.append(Vect_ad_key.iloc[j-2])
            list_next_key.append(0)
            list_2next_key.append(0)
        else :
            list_prev_word.append(Vect_ad_word.iloc[j-1])
            list_2prev_word.append(Vect_ad_word.iloc[j-2])
            list_next_word.append(Vect_ad_word.iloc[j+1])
            list_2next_word.append(Vect_ad_word.iloc[j+2])
            
            list_prev_key.append(Vect_ad_key.iloc[j-1])
            list_2prev_key.append(Vect_ad_key.iloc[j-2])
            list_next_key.append(Vect_ad_key.iloc[j+1])
            list_2next_key.append(Vect_ad_key.iloc[j+2])

On complete la dataframe

In [9]:
df["2prev_word"]=list_2prev_word
df["2prev_key"]=list_2prev_key

df["prev_word"]=list_prev_word
df["prev_key"]=list_prev_key

df["next_word"]=list_next_word
df["next_key"]=list_next_key

df["2next_word"]=list_2next_word
df["2next_key"]=list_2next_key

In [10]:
df.loc[df['Ad#']==482,:].head() # On regarde les features de l'annonce 482

,Ad#,Words,Pos,Tag,is_small_word,is_number,is_first_letter_upper,is_all_upper,symbole_in_word,is_number_and_letter,is_key_word,2prev_word,2prev_key,prev_word,prev_key,next_word,next_key,2next_word,2next_key
41057,482,24,"[0, 2]",B-ADRESSE,1,1,0,0,0,0,0,__Start2__,0,__Start1__,0,rue,1,du,0
41058,482,rue,"[3, 6]",I-ADRESSE,1,0,0,0,0,0,1,__Start1__,0,24,0,du,0,Capitaine,0
41059,482,du,"[7, 9]",I-ADRESSE,1,0,0,0,0,0,0,24,0,rue,1,Capitaine,0,Ferber-,0
41060,482,Capitaine,"[10, 19]",I-ADRESSE,0,0,1,0,0,0,0,rue,1,du,0,Ferber-,0,Copropriété,0
41061,482,Ferber-,"[20, 27]",O,0,0,1,0,1,0,0,du,0,Capitaine,0,Copropriété,0,de,0


In [11]:
df.isnull().sum() # On verifie qu'il n'y a pas de NA

Ad#                      0
Words                    0
Pos                      0
Tag                      0
is_small_word            0
is_number                0
is_first_letter_upper    0
is_all_upper             0
symbole_in_word          0
is_number_and_letter     0
is_key_word              0
2prev_word               0
2prev_key                0
prev_word                0
prev_key                 0
next_word                0
next_key                 0
2next_word               0
2next_key                0
dtype: int64

# III/ Classification avec le modele CRF

Pour utiliser le modele CRF, on doit modifier notre base de donnée et créer des dictionnaire pour pouvoir mettre notre dataframe en entrée du modele

### A/ Creation base de donnée pour CRF

In [12]:
# Pour chaque mot on crée un dictionaire à partir de la dataframe existante
def word2features(df_one_ad, i):
    df_one_word = df_one_ad.iloc[i]
    features = {
        'bias': 1.0,
        'word.lower()': df_one_word["Words"].lower(),
        'is_small_word': bool(df_one_word["is_small_word"]),
        'is_number': bool(df_one_word["is_number"]),
        'is_first_letter_upper': bool(df_one_word["is_first_letter_upper"]),
        'is_all_upper': bool(df_one_word["is_all_upper"]),
        'symbole_in_word': bool(df_one_word["symbole_in_word"]),      
        'is_number_and_letter': bool(df_one_word["is_number_and_letter"]),
        'is_key_word': bool(df_one_word["is_key_word"]),
        '2prev_word': df_one_word["2prev_word"].lower(),
        '2prev_key': bool(df_one_word["2prev_key"]),
        'prev_word': df_one_word["prev_word"].lower(),
        'prev_key': bool(df_one_word["prev_key"]), 
        'next_word': df_one_word["next_word"].lower(),
        'next_key': bool(df_one_word["next_key"]), 
        '2next_word': df_one_word["2next_word"].lower(),
        '2next_key': bool(df_one_word["2next_key"]), 
    }
    return features

# Vecteur contenant les dictionnaires d'une annonce 
def ad2features(df_one_ad):
    return [word2features(df_one_ad, i) for i in range(len(df_one_ad))]

# Return le tag d'un mot
def word2tags(df_one_ad, i):
    return df_one_ad.iloc[i]["Tag"]

# Vecteur de tags d'une annonce
def ad2tags(df_one_ad):
    return [word2tags(df_one_ad, i) for i in range(len(df_one_ad))]

Number_ad=df['Ad#'].iloc[-1]

# Base de données pour CRF
X_crf=list() 
y_crf=list() # Target 
for i in range(Number_ad):
    X_crf.append(ad2features(df.loc[df['Ad#']==i+1,:]))
    y_crf.append(ad2tags(df.loc[df['Ad#']==i+1,:]))
    

In [13]:
X_crf[0][0:2] # On observe la base de donnée CRF pour les deux premiers mots 

[{'bias': 1.0,
  'word.lower()': 'situé',
  'is_small_word': False,
  'is_number': False,
  'is_first_letter_upper': True,
  'is_all_upper': False,
  'symbole_in_word': False,
  'is_number_and_letter': False,
  'is_key_word': True,
  '2prev_word': '__start2__',
  '2prev_key': False,
  'prev_word': '__start1__',
  'prev_key': False,
  'next_word': 'à',
  'next_key': False,
  '2next_word': '6',
  '2next_key': False},
 {'bias': 1.0,
  'word.lower()': 'à',
  'is_small_word': True,
  'is_number': False,
  'is_first_letter_upper': False,
  'is_all_upper': False,
  'symbole_in_word': False,
  'is_number_and_letter': False,
  'is_key_word': False,
  '2prev_word': '__start1__',
  '2prev_key': False,
  'prev_word': 'situé',
  'prev_key': True,
  'next_word': '6',
  'next_key': False,
  '2next_word': 'stations',
  '2next_key': False}]

In [14]:
y_crf[0][0:2] # On observe la base de donnée CRF pour les deux premiers mots 

['O', 'O']

### B/ Apprentissage et Validation du modele CRF

Creation de la base de train et de test (proportion 2/3, 1/3)

In [15]:
X_crf_train, X_crf_test, y_crf_train, y_crf_test = train_test_split(X_crf,y_crf,test_size=0.33, random_state=12)

#### 1/ Apprentissage et Validation simple du modele CRF

On entraine le modele avec la base de train.

In [16]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_crf_train, y_crf_train)

Wall time: 1min 5s


In [17]:
# On affiche les différents modeles que l'on doit recuperer
labels = list(crf.classes_)
labels.remove('O')
labels

['B-ADRESSE',
 'I-ADRESSE',
 'B-N_ETAGE',
 'B-AVEC_ASCENSEUR',
 'I-AVEC_ASCENSEUR',
 'B-N_PIECES',
 'B-N_CHAMBRES',
 'B-STOCKAGE',
 'B-PARKING',
 'B-DATE_DISPO',
 'I-VILLE',
 'B-TRANSPORTS_PROXIMITE',
 'I-TRANSPORTS_PROXIMITE',
 'B-TYPE_CHAUFFAGE',
 'B-M2',
 'B-EXTERIEUR',
 'B-QUARTIER',
 'B-VILLE',
 'B-LOYER_CC',
 'I-DATE_DISPO',
 'B-HONORAIRE',
 'I-QUARTIER',
 'I-N_PIECES',
 'B-TYPE_LOCATION',
 'B-ANNEE_CONSTRUCTION',
 'B-LOYER_HC',
 'B-CHARGES_LOCATAIRE_MOIS',
 'B-DEPOT_GARANTIE',
 'I-N_ETAGE',
 'I-LOYER_HC',
 'I-ANNEE_CONSTRUCTION',
 'I-DEPOT_GARANTIE',
 'B-COPROPRIETE',
 'I-HONORAIRE',
 'B-CODE_POSTAL',
 'I-LOYER_CC',
 'I-M2',
 'I-TYPE_CHAUFFAGE',
 'I-TYPE_LOCATION',
 'I-COPROPRIETE',
 'I-STOCKAGE',
 'I-PARKING',
 'I-CHARGES_LOCATAIRE_MOIS']

On predit les classes sur la base de test

In [18]:
y_pred = crf.predict(X_crf_test)
metrics.flat_f1_score(y_crf_test, y_pred, 
                      average='weighted', labels=labels)

# WARNING EXPLANATION some labels in y_true don't appear in y_pred
# This means that there is no F-score to calculate for this label, and thus the F-score for this case is considered to be 0.0. Since you requested an average of the score, you must take into account that a score of 0 was included in the calculation, and this is why scikit-learn is showing you that warning.

C:\Users\sujiv\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\sujiv\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.8412282068598782

On observe une accuracy proche de 85%, ce qui pas mal vu le nombre de classes à predire. 

Nous allons calculer la precision, le recall et le f1-score pour chaque classe.

In [19]:
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_crf_test, y_pred, labels=sorted_labels, digits=3
))

C:\Users\sujiv\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\sujiv\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                          precision    recall  f1-score   support

               B-ADRESSE      0.957     0.800     0.871        55
               I-ADRESSE      0.919     0.771     0.839       118
    B-ANNEE_CONSTRUCTION      0.950     0.679     0.792        28
    I-ANNEE_CONSTRUCTION      1.000     0.600     0.750         5
        B-AVEC_ASCENSEUR      0.754     0.776     0.765        67
        I-AVEC_ASCENSEUR      0.767     0.868     0.814        53
B-CHARGES_LOCATAIRE_MOIS      0.895     0.919     0.907        37
I-CHARGES_LOCATAIRE_MOIS      0.000     0.000     0.000         0
           B-CODE_POSTAL      0.833     0.833     0.833         6
           B-COPROPRIETE      0.850     1.000     0.919        17
           I-COPROPRIETE      0.000     0.000     0.000         0
            B-DATE_DISPO      0.816     0.705     0.756        44
            I-DATE_DISPO      0.903     0.651     0.757        43
        B-DEPOT_GARANTIE      1.000     0.955     0.977        22
        I

Pour tout les F1_score inferieur à 0.75 et un support superieur à 10, nous allons regarder la matrice de confusion pour éventuellement trouver des features à rajouter. 

Nous allons regarder B-LOYER-CC (l18), B-QUARTIER (l31), I-QUARTIER (l32) et B-TYPE_LOCATION (l39)

In [27]:
flat_list_y_test = [item for sublist in y_crf_test for item in sublist]
flat_list_y_pred= [item for sublist in y_pred for item in sublist]
conf_mat=confusion_matrix(flat_list_y_test, flat_list_y_pred, labels=sorted_labels)
print(conf_mat[18])
print(conf_mat[31])
print(conf_mat[32])
print(conf_mat[39])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0 13  0  4  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0 16  3  0  0  0  0  0  0  0  0  0  0]
[ 0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  3 11  0  0  0  0  0  0  0  0  1  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 33  0  0  0]


In [34]:
nb_good_pred=0
sum_line=0
for i in range(len(conf_mat)):
    nb_good_pred+=conf_mat[i,i]
    sum_line+=sum(conf_mat[i])
print("Nombre d'erreur en prevoyant 'O' ",2067-sum_line)
print("Nombre d'erreur en prevoyant un autre label que 'O' ",sum_line-nb_good_pred)

Nombre d'erreur en prevoyant 'O'  285
Nombre d'erreur en prevoyant un autre label que 'O'  68


Lorsque le tag est mauvais c'est souvent dû au fait qu'on predit un label 'O' (ce n'est pas indiqué sur la matrice de confusion mais pour calculer le nombre de mauvaises prédictions en 'O', on a juste à faire support - somme element sur la ligne). Nous avons deja des features mots cles, il est difficile de trouver encore d'autres features pour differencier 'O' et les 'vrais' tag.

#### 2/ Apprentissage après optimisation des hyperparametres et k-cross Validation sur le train et validation du modele CRF sur le test

##### Optimisation Hyperparametres

ATTENTION : Le prochain chunk met du temps à s'executer

On utilise la fonction RandomizedSearchCV pour optimiser les hyperparametres. 
Remarque : en argument de cette fonction, on peut mettre en entrée le nombre k, utilisé pour le k-cross Validation.
Nous avons choisi k=3, on pourrait mettre un plus grand nombre mais vu le temps elevé pour l'execution du code, on se contente de k=3.

In [21]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_crf_train, y_crf_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 34.7min finished


Wall time: 35min 27s


Meilleur paramètre

In [22]:
print(rs.best_estimator_)
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None,
  c1=0.37830775726188476, c2=0.0020786765046253786,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)
best params: {'c1': 0.37830775726188476, 'c2': 0.0020786765046253786}
best CV score: 0.8520192103514235


##### Validation sur le test avec les meilleurs paramètres

In [23]:
crf = rs.best_estimator_
y_pred = crf.predict(X_crf_test)
print(metrics.flat_classification_report(
    y_crf_test, y_pred, labels=sorted_labels, digits=3
))

                          precision    recall  f1-score   support

               B-ADRESSE      0.936     0.800     0.863        55
               I-ADRESSE      0.920     0.780     0.844       118
    B-ANNEE_CONSTRUCTION      0.950     0.679     0.792        28
    I-ANNEE_CONSTRUCTION      1.000     0.600     0.750         5
        B-AVEC_ASCENSEUR      0.768     0.791     0.779        67
        I-AVEC_ASCENSEUR      0.770     0.887     0.825        53
B-CHARGES_LOCATAIRE_MOIS      0.897     0.946     0.921        37
I-CHARGES_LOCATAIRE_MOIS      0.000     0.000     0.000         0
           B-CODE_POSTAL      0.833     0.833     0.833         6
           B-COPROPRIETE      0.850     1.000     0.919        17
           I-COPROPRIETE      0.000     0.000     0.000         0
            B-DATE_DISPO      0.737     0.636     0.683        44
            I-DATE_DISPO      0.824     0.651     0.727        43
        B-DEPOT_GARANTIE      1.000     0.955     0.977        22
        I

Les résultats sont similaires même apres l'optimisation.

# IV/ Interpretation avec le modele CRF

On regarde les transitions possibles entre les labels (plus le nombre est grand plus la possibilité de transition entre ces deux labels est grande).

In [24]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
I-ADRESSE -> I-ADRESSE 7.721319
B-ADRESSE -> I-ADRESSE 7.405639
I-QUARTIER -> I-QUARTIER 6.573868
B-QUARTIER -> I-QUARTIER 6.555793
I-N_ETAGE -> I-N_ETAGE 6.444019
B-DATE_DISPO -> I-DATE_DISPO 6.338790
B-AVEC_ASCENSEUR -> I-AVEC_ASCENSEUR 6.208596
I-VILLE -> I-VILLE 6.098950
I-M2   -> I-M2    5.695440
B-HONORAIRE -> I-HONORAIRE 5.531093
B-LOYER_CC -> I-LOYER_CC 5.520942
I-ANNEE_CONSTRUCTION -> I-ANNEE_CONSTRUCTION 5.520584
B-DEPOT_GARANTIE -> I-DEPOT_GARANTIE 5.518223
I-TYPE_LOCATION -> I-TYPE_LOCATION 4.933606
I-TRANSPORTS_PROXIMITE -> I-TRANSPORTS_PROXIMITE 4.913720
I-AVEC_ASCENSEUR -> I-AVEC_ASCENSEUR 4.890893
I-DATE_DISPO -> I-DATE_DISPO 4.838904
B-CODE_POSTAL -> B-VILLE 4.444149
B-ANNEE_CONSTRUCTION -> I-ANNEE_CONSTRUCTION 4.419642
B-TRANSPORTS_PROXIMITE -> I-TRANSPORTS_PROXIMITE 4.305080

Top unlikely transitions:
B-VILLE -> B-N_PIECES -0.001894
I-QUARTIER -> B-QUARTIER -0.191477
O      -> I-M2    -0.228010
B-DATE_DISPO -> B-DATE_DISPO -0.248292
B-AVEC_ASC

Au vu de la convention IOB qu'on suit, les resultats sont coherents.

On affiche les features les plus communs et les moins communs.

In [25]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
11.368693 B-TRANSPORTS_PROXIMITE word.lower():tram
10.292584 B-STOCKAGE word.lower():cave
9.890270 B-M2     next_word:m²
9.806920 B-TRANSPORTS_PROXIMITE word.lower():tramway
9.653988 B-TRANSPORTS_PROXIMITE word.lower():rer
9.351851 B-N_CHAMBRES next_word:chambres
9.108145 B-QUARTIER word.lower():centre-ville
9.052081 B-STOCKAGE word.lower():box
8.995772 B-N_PIECES next_word:pièces
8.899276 B-COPROPRIETE word.lower():copropriété
8.392942 B-TYPE_LOCATION word.lower():vide
8.211898 B-VILLE  word.lower():issy
8.107117 B-TRANSPORTS_PROXIMITE word.lower():métro
7.808568 B-PARKING word.lower():garage
7.648257 B-PARKING word.lower():stationnement
7.498618 B-DEPOT_GARANTIE 2prev_word:garantie
7.492127 B-ADRESSE word.lower():impasse
7.474122 B-PARKING word.lower():parking
7.396491 B-AVEC_ASCENSEUR word.lower():ascenseur
7.271364 B-EXTERIEUR word.lower():balcon
7.147166 B-M2     prev_word:d'environ
7.046329 B-DATE_DISPO word.lower():immédiatement
6.985873 B-EXTERIEUR word.lower():vé

On affiche les poids des transition et les top features pour chaque classe.

In [26]:
 eli5.show_weights(crf, top=10) # Currently ELI5 allows to explain weights and predictions of scikit-learn classifier

From \ To,O,B-ADRESSE,I-ADRESSE,B-ANNEE_CONSTRUCTION,I-ANNEE_CONSTRUCTION,B-AVEC_ASCENSEUR,I-AVEC_ASCENSEUR,B-CHARGES_LOCATAIRE_MOIS,I-CHARGES_LOCATAIRE_MOIS,B-CODE_POSTAL,B-COPROPRIETE,I-COPROPRIETE,B-DATE_DISPO,I-DATE_DISPO,B-DEPOT_GARANTIE,I-DEPOT_GARANTIE,B-EXTERIEUR,B-HONORAIRE,I-HONORAIRE,B-LOYER_CC,I-LOYER_CC,B-LOYER_HC,I-LOYER_HC,B-M2,I-M2,B-N_CHAMBRES,B-N_ETAGE,I-N_ETAGE,B-N_PIECES,I-N_PIECES,B-PARKING,I-PARKING,B-QUARTIER,I-QUARTIER,B-STOCKAGE,I-STOCKAGE,B-TRANSPORTS_PROXIMITE,I-TRANSPORTS_PROXIMITE,B-TYPE_CHAUFFAGE,I-TYPE_CHAUFFAGE,B-TYPE_LOCATION,I-TYPE_LOCATION,B-VILLE,I-VILLE
O,2.742,0.945,-2.68,0.752,-1.994,1.883,-2.599,1.053,0.0,0.0,1.643,0.0,2.353,-1.889,0.784,0.0,2.549,1.644,0.0,0.468,0.0,1.07,0.0,2.325,-0.228,0.98,1.67,-1.7,2.213,0.0,1.966,0.0,1.931,-1.101,1.833,0.0,2.524,-2.267,1.672,0.0,1.934,-0.369,2.079,0.0
B-ADRESSE,0.0,0.0,7.406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-ADRESSE,0.235,0.0,7.721,0.0,0.0,0.0,0.0,0.0,0.0,0.127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-ANNEE_CONSTRUCTION,0.33,0.0,0.0,0.0,4.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.313,-0.736,0.0,0.0,0.0,0.0,0.0,0.0
I-ANNEE_CONSTRUCTION,-0.0,0.0,0.0,0.0,5.521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-AVEC_ASCENSEUR,-0.324,0.0,0.0,0.0,0.0,0.0,6.209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-AVEC_ASCENSEUR,0.296,0.0,0.0,0.0,0.0,0.0,4.891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-CHARGES_LOCATAIRE_MOIS,0.294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-CHARGES_LOCATAIRE_MOIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-CODE_POSTAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.444,0.0
